In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 75
l = 6
num_filter = 32
compression = 0.5
dropout_rate = 0

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(X_train)
img_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
steps = X_train.shape[0] // batch_size

In [8]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('elu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3),kernel_regularizer=regularizers.l2(1e-4),
                                   use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('elu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1),kernel_regularizer=regularizers.l2(1e-4),
                                      use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('elu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [9]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 32)   0           ['batch_normalization[0][0]']

In [11]:
print(len(model.layers))

115


In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001,decay=1e-6),
              metrics=['accuracy'])

In [13]:
filepath = 'my_best_model.epoch{epoch:02d}-accuracy{val_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

In [14]:
%load_ext tensorboard

In [15]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
model.fit(img_gen,
          steps_per_epoch=steps,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint,tensorboard_callback])

Epoch 1/75
781/781 [==============================] - ETA: 0s - loss: 1.5504 - accuracy: 0.4597 - ETA - ETA: 41s - loss: 1.88 - ETA: 40s - loss: 1.8576 - accu - ETA: 39s - loss: 1.8444 - accuracy: 0 - ET - ETA: 36s - loss: 1.80 - ETA: 35s - loss: 1.7944 - acc - ETA: 34s - loss: 1.7850 - accuracy: 0.3 - ETA: 34s - loss: 1.7833 - accuracy: 0.3 - ETA: 33s - loss: 1. - ETA: 32s - loss: 1.7691 - accuracy - ETA: 31s - loss: 1.7636 - ETA: 30s - loss: 1.7506 - accuracy: - ETA: 30s - loss: 1.7452 - accuracy - ETA: 29s - loss: 1.7400 - accurac - ETA: 28s - loss: 1.7 - ETA:   - ETA: 19s - loss: 1.6661 - accuracy:  - ETA: 19s - loss: 1.6622 - accuracy: 0.416 - ETA: 19s - loss: 1.6616 - accurac - ETA: 18s - loss - ETA: 16s - loss: 1.6470 - accuracy: - ETA: 16s - loss: 1.6434 - accuracy: 0. - ETA: 16s -  - ETA:  - ETA - ETA: 2s - - ETA: 1s - loss: 1.5564 - accuracy: 0. - ETA: 0s - loss: 1.555 - ETA: 0s - loss: 1.5527 - accura
Epoch 00001: val_accuracy improved from -inf to 0.51650, saving model to m

781/781 [==============================] - ETA: 0s - loss: 0.8191 - accuracy: 0.7436 - ETA: 49s -  - ETA: 47s - loss: 0.8406 - accuracy:  - ETA: 47s - loss: 0. - ETA: - ETA: 39s - loss: 0.8210 - accuracy: 0 - ETA: 39s - - ETA: 37s - l - ETA: 35s - loss: 0.823 - ETA: 33s - loss: - ETA: 32s - - ETA: 30s - loss: 0.8210 - accuracy: 0.740 - E - ETA: 27s - loss: 0.8245 - accuracy - ETA: 26s - loss: 0.8235 - accuracy: - ETA: 26s - loss: 0.8230  - ETA: 25s - loss: 0.8221 - ac - ETA: 24s -  - ETA: 22s - loss: 0.8230 - accuracy: 0.7 - ETA: 22s - loss: 0.8223 - accuracy:  - E - ETA: 18s - loss: 0.8241 - accurac - ETA: 18s - loss: 0.8234 - accuracy: 0.7 - ETA: 18s -  - ETA: - ETA: 13s - loss: 0.8238  - ETA: 12s - loss - ETA: 8s - loss: 0 - ETA: 8s - loss: 0.8192 - accura - ETA: 7s - loss: 0.8187 - accu - ETA: 7s - loss: 0.8191 - accuracy: 0.74 - ETA: 7s - loss: 0.8 - ETA: 6s - loss: 0.8191 - accuracy: 0. - ETA: 6s - los - ETA
Epoch 00007: val_accuracy improved from 0.73460 to 0.74400, saving model

781/781 [==============================] - ETA: 0s - loss: 0.6746 - accuracy: 0.80107 - ETA: 48s - loss: 0.7 - ETA: 46s - loss: 0.6710 - accu - ETA: 45s - loss: 0 - ETA:  - ETA: 41s - loss: 0.67 - ETA: 40s - loss: 0.6764 -  - ETA: 36s - loss: 0.6763 - a - ETA: 35s - loss: 0.6737 - accuracy: 0.802 - ETA: 35s - loss: 0.6746 - accuracy: 0.8 - ETA: 35s - loss: 0.6732 - accurac - ETA: 34s - loss: 0.6738 - a - ETA: 33s - loss: 0.6755 - accuracy: 0.80 - ETA: 33s - loss: 0.6756 - ET - ETA: 26s - loss: 0.67 - ETA: 25s - loss: 0.6752 - ETA: 24s - loss: 0.6740 - accuracy: 0.801 - ETA: 24s - loss:  - ETA: 22s - loss: 0.6730 - accuracy: 0.8 - ETA: 22s - loss: 0.6738 - accuracy: 0.80 - ETA: 22s - loss: 0.6732 - accurac - ETA: 21s - loss: 0.6729 - acc - ETA: 20s - lo - ETA: 15s - loss:  - ETA: 14s - loss: 0.675 - ETA: 12s - loss: 0.6763 - accur - ETA: 12s - loss:  - ETA: 10s - loss: 0.6764 - accuracy:  - ETA: 9s  - ETA: 4s - loss: 0.6759 - accuracy - ETA: 4s - los - ETA: 3s - loss: 0.6762 - accu
Epoc

781/781 [==============================] - ETA: 0s - loss: 0.6076 - accuracy: 0.8258 - ETA: 42s - loss: 0.6041 - accuracy: 0.82 - ETA: 41s - loss: 0.6033 - - ETA: 37s - los - ETA: 35s - loss: 0.6066 - accuracy: 0.8 - ETA: 35s - loss: 0.6065 - accuracy: 0.8 - ETA: 35s - loss: 0.6045 -  - ETA: 31s - loss: 0.6050 - accuracy: 0.8 - E - ETA: 28s - loss: 0.6054 - accuracy: - ETA: 28s - loss: 0.6051 - a - ETA: 27s - loss: 0.6060 - accuracy - ETA: 26s - loss: 0.6060 - ETA: 25s - loss: 0.6066 - accuracy: 0.8 - ETA: 25s - loss: 0.605 - ETA: 23s - loss: 0. - ETA: 22s  - ETA: 19s - loss: 0.6060 - accuracy: - ETA: 19s - loss: 0.6047 - accuracy: 0.82 - ETA: 19s - loss: 0.6047 - acc - ETA: 18 - ETA: 16s - loss: 0.6051 - accu - ETA: 1 - ETA: 6s - loss: 0 - ETA: 3s - - ETA: 0s - loss: 0.607
Epoch 00021: val_accuracy improved from 0.81270 to 0.81910, saving model to my_best_model.epoch21-accuracy0.82.hdf5
781/781 [==============================] - 57s 73ms/step - loss: 0.6076 - accuracy: 0.8258 - val_lo

781/781 [==============================] - ETA: 0s - loss: 0.5631 - accuracy: 0.8453 - ETA: 48s - loss: 0.6326 - ETA: 48s - loss: 0.5631 - accuracy: 0 - ETA: 48s - loss: 0.5689 - accurac - ETA: 47s - l - ETA: 45s - loss: 0.5643 - - ETA: 41s - loss: 0.5724 - accuracy: 0 - ETA: 41s  - ETA: 39s - loss: 0.5690 - accu - ETA: 38s - loss: 0.5648 - accurac - ETA: 38s - loss: 0.5640 - accuracy:  -  - ETA: 32s - loss: 0.5560  - ETA: 31s - loss: - ETA: 29s - loss: 0.5565 - accu - ETA: 28s - loss: 0.5559 - accuracy: 0.846 - ETA: 28s - loss: 0.5556 - a - ETA: 24s - loss: 0 - ETA: 22s - loss: 0.5566 - - ETA: 21s - loss: 0.5562 - accuracy:  - ETA: 21s - loss: 0.5562 - accuracy: 0 - ETA: 21s - loss: 0.5562 - a - ETA: 20s - loss: 0.5574 - ETA:  - ETA: 10s - loss: 0.5612 - accuracy: - ETA: 10s - loss: 0.5598 - accuracy: - ETA: 9s - l - ETA: 8s - loss: 0.559 - ETA - ETA: 4s - loss: 0.5 - ETA: 3s - loss: 0.5631 - 
Epoch 00028: val_accuracy did not improve from 0.83460
781/781 [============================

...

781/781 [==============================] - ETA: 0s - loss: 0.4952 - accuracy: 0.8700 - ETA: 49s - loss: - ETA: 48s - loss: 0.45 - ETA: 47s - loss: 0.4597 - accuracy - ETA: 46s - loss: 0.4662 - accurac - ETA: 46s - loss: 0.4629 - accuracy: 0.8 - ETA: 46s - loss: 0.4654 - a - ETA: 44s - loss: 0.4712 - acc - ETA: 44s - loss: 0.47 - ETA: 3 - ETA: 31s - loss: 0.4866 - accurac - ETA: 30s - loss: 0.4859 -  - ETA: 29s - loss:  - ETA: 28s - loss: 0.4898 - accuracy - ETA: 27s - los - ETA: 25s - loss: 0.4896 - accuracy: 0.87 - ETA: 25s - loss: 0.4894 - accurac - ETA: 25s - loss: 0.4901 - accuracy: 0.87 - ETA: 24s - loss - ETA: 23s - loss: 0.4899 - accuracy: 0.870 - ETA:  - ETA: 20s - loss: 0.4 - ETA: 19s - loss: 0.4 - ETA: 17s - loss: 0.4920 - accuracy:  - ETA: 17s - loss: 0.4923 - accuracy: - ETA: 16s - loss: 0.4926 - accuracy - ETA: 16s - loss: 0.4937 - accuracy - ETA: 15s - loss: 0.4940 -   - ETA: 9s - loss: 0.492 - ETA: 5s - loss: 0.4930 - accuracy - ETA: 5s - loss: 0.4928 -  - ETA: 4s - loss

781/781 [==============================] - ETA: 0s - loss: 0.4831 - accuracy: 0.8747 - ETA: 51s - loss: 0.4806 - accuracy: 0.87 - ETA: 51s - ETA: 48s - loss: 0.4553 - accuracy: 0.88 - ETA: 48s - loss: 0. - ETA: 46s - - ETA: 44s - loss: 0.4643 - accura - ETA: 43s - loss: 0.4 - ETA: 42s - l - ETA: 40s - l - ETA: 38s - loss: 0.4670 - accuracy: 0. - ETA: 38s - loss: 0.4656 - accuracy: 0 - ETA: 38s - loss: 0.4666 - accuracy:  - ETA: 37s - loss:  - ETA: 35s -  - ETA: 30s - loss: - ETA: 29s - loss: 0.4758 - accuracy: 0.878 - ETA: 29s - loss: 0.4756 - accur - ETA: 28s -  - ETA: 26s - loss: 0.4778 - accur - ETA: 25s - loss: 0. - E - ETA: 21s - loss: 0.4803 - accuracy:  - ETA: 20s - loss: 0.4795 - accu - ETA: 19s - loss: 0.4806 - accuracy: 0.875 - ETA: 19s - loss: 0.4807 - accuracy: 0. - ETA: 19s - loss: 0.480 - ETA: 18s - loss: 0.4804 - acc - ETA: 17s - loss: 0.4808 - accuracy: 0.87 - ETA: 17s - los - ETA: 15s - loss: 0.4815 - acc - ETA: 14s - loss: 0.48 - ETA: 13s - loss: 0.4818 - accuracy: 0.

781/781 [==============================] - ETA: 0s - loss: 0.4717 - accuracy: 0.8800 - ETA: 50s - loss: 0.425 - ETA: 49s - loss: 0.4621 - ac - ETA: 48s - loss: 0.4598 - accura - ETA: 48s  - ETA: 45s - loss: 0.4673 - acc - ETA: 45s - loss: 0.4644 - accuracy - ETA: 44s - loss: 0.4642 - accuracy: 0.88 - ETA: 44s - loss: 0.4645 - accura - ETA: 37s - loss: 0.4695 - accuracy: 0 - ETA: 37s - loss: - ETA: 35s - ETA: 30s - loss: 0.4702 - accuracy: 0. - ETA: 30s - loss: 0.4694 - accuracy: 0 - ETA: 30s - loss: 0.4702 - accu - ETA: 26s - loss: 0.4687 - accuracy: 0 - ETA: 26s - loss: 0.468 - ETA: 24s - loss: 0.4689 - accur - ETA: 24s - loss: 0.4699 - accuracy:  - ETA: 23s - loss: 0.4701 - accuracy: 0 - ETA: 23s - loss: 0.4702 - acc - ETA: 22s - loss: 0.4700 - accuracy: 0.881 - ETA - ETA: 19s - loss: 0.4684 - acc - ETA: 18s - ETA: 16s - loss: 0.4730 - accuracy: 0 - ETA: 16s - loss: 0.4729 - accuracy - ETA: 15s - loss: 0.4728 - accu - ETA: 15s - loss: 0.4723 - accuracy: 0.88 - ETA: 14s - loss: 0.4719

781/781 [==============================] - ETA: 0s - loss: 0.4587 - accuracy: 0.8824 - ETA: 50s - loss: 0.4323 - a - ETA: 48s - loss: 0 - ETA: 47s - loss: 0.4444 - accuracy: 0 - ETA: 47s - l - ETA: 45s - loss: 0.4393 - accur - ETA: 44s - loss:  - ETA: 42s - loss: 0.4419 - accur - ETA: 41 - ETA: 30s - loss: 0.450 - ETA: 28s - loss: 0.4502 - acc - ETA: 28s  - ETA: 22s - loss: 0.4544 -  - ETA: 21s -  - ETA: - ETA: 17s - loss: 0.4540 - accurac - ETA: 16s - loss: 0.455 - ETA: 12s - loss: 0.4565 - accuracy: 0.883 - ETA: 12s - loss: 0.4565 - accuracy: 0.88 - ETA: 12s - loss: 0.4565 - acc - ETA: 11s - l - ETA: 0s - loss: 0
Epoch 00066: val_accuracy did not improve from 0.86510
781/781 [==============================] - 56s 72ms/step - loss: 0.4587 - accuracy: 0.8824 - val_loss: 0.6347 - val_accuracy: 0.8394
Epoch 67/75
781/781 [==============================] - ETA: 0s - loss: 0.4601 - accuracy: 0.8826 - ETA: 49s - loss: 0.4518 - accuracy:  - ETA: 49s - loss: 0.4418 - accuracy: - ETA: 49s - lo

781/781 [==============================] - ETA: 0s - loss: 0.4491 - accuracy: 0.8874 - ETA: 46s - loss: 0.4543 - acc - ETA: 45s - loss: 0.4559 - accuracy: 0. - E - ETA: 43s -  - ETA: 40s - loss: 0.4582 - accuracy: - ETA: 40s - loss: 0.4569 - accur - ETA: 39s - loss: 0.4558 - accurac - ETA: 39s - loss: 0.4569 - accuracy: - ETA: 35s - loss: 0.4524 - accuracy:  - ETA: 35s - loss: 0.4 - ETA: 33s - loss:  - ETA: 31s - loss: 0.4560 - acc - ETA: 31s - loss: 0.4552 - accuracy: 0.88 - ETA: 31s - loss: 0.4554 - ac - ETA: 30s - loss: 0.4548 - accura - ETA: 29s - loss: 0.4545 - accuracy: 0.88 - ETA: 29s - loss: 0.4545 - accura - ETA: 28s - lo - ETA: 26s - loss: 0.4513 - accuracy: - ETA: 26s - loss: 0.4523 - accura - ETA: 25s - loss: 0.45 - ETA: 24s - loss: 0.4505 - accurac - ETA: 23s - loss: 0.4502 - accuracy: 0.8 - ETA: 23s - loss: 0.4498 - accur - ETA: 19s - loss: 0.4492 - accur - ETA: 19s - loss: 0.4492 - a - ETA: 18s - loss: 0.4 - ETA: 16s - loss: 0.4487 - accuracy: 0.888 - ETA: 16s - loss: 0.

In [17]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7780), started 10:43:18 ago. (Use '!kill 7780' to kill it.)

In [18]:
model.save_weights("model_rms_80ep.h5")

In [19]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.0005,decay=1e-6),
              metrics=['accuracy'])

In [21]:
model.fit(img_gen,
          steps_per_epoch=steps,
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint,tensorboard_callback])

Epoch 1/100
781/781 [==============================] - ETA: 0s - loss: 0.3924 - accuracy: 0.9060 - ETA: 51s - loss: 0.4297 - - ETA: 50s - loss: 0.4249 - accuracy: 0.895 - ETA: 51s - loss: - ETA: 48s - loss: 0.4061 - accuracy:  - ETA: 48s - loss: 0.4034 - accuracy:  - ETA: 47s - loss: 0.4050 - accurac - ETA: 46s - loss: 0.4065 - accur - ETA: 45s - loss: 0.4042 - ac - ETA: 44s - loss: 0.4060 - accurac - ETA: 44s - loss: 0.4067 - - ETA: 36s - loss: 0.4010 - accura - ETA: 36s - loss: 0.3998 - ac - ETA: 35s - ETA: 32s - loss: 0.3970 - accuracy:  - ETA: 32s - loss: 0.3965 - accuracy:  - ETA: 32s - loss: 0.3964 - ETA: 30s - loss: - ETA: 29s - loss: 0.3951 - accuracy: 0.9 - ETA: 28s - loss: 0.3952 - accuracy: 0.9 - ETA: 28s - loss:  - ETA: 27s - loss: 0.3948 - accura - ETA: 26s - loss: 0.3940 - accuracy: 0 - ETA: 26s - loss: 0.3947 - accur - ETA: 25s - loss: 0.3949 - acc - ETA: 24s - loss: 0.39 - ETA: 23s - loss: 0.3947 -  - ETA: 22s - loss: 0.3954 - accuracy: 0. - ETA: 21s - loss: 0.3958 - ac

781/781 [==============================] - ETA: 0s - loss: 0.3594 - accuracy: 0.9139 - ETA: 47s - loss: 0.3704 - - ETA: 46s - loss - ETA: 44s - loss: 0.3546 - accuracy: - ETA: 44s - loss:  - ETA: 42s - loss: 0.3503 - a - ETA: 41s - loss: 0.3477 - - ETA: 40s - l - ETA: 39s - loss: 0.3540 - acc - ETA: 38s - loss: 0.3567 - accura - ETA: 37s - loss: 0.3560 - accurac - ETA: 37s -  - ETA: 34s - loss: 0.3612 - accuracy: 0.913 - ETA: 34s - loss: 0.3614  - ETA: 33s - loss: - ETA: 31s - loss: 0.3605 - accu - ETA: 31s - loss: 0.3596 - accur - ETA: 30s - loss: 0.3580 - accuracy: 0. - ETA: 30s - loss: 0.3582 - ac - ETA: 29s - loss: 0.3583 - accuracy: - ETA: 28s - loss: 0.3574 - accura - ETA: 28s - loss: 0.3575 - accurac - ETA: 27s - loss: 0.3593 - accuracy: 0.914 - E - ETA: 19s - loss: 0.3587 - accuracy: 0.91 - ETA: 18s - loss: 0.3588 - accuracy: 0.91 - ETA: 18s - loss: 0.3588 - accuracy - ETA: 18s - loss:  - ETA: 16s - loss: 0.359 - ETA: 15s - loss: 0.3593 - accuracy - ETA: 14s - loss: 0.3594 - ac

781/781 [==============================] - ETA: 0s - loss: 0.3465 - accuracy: 0.9151 - ETA: 51s - loss: 0.3191 - accuracy: 0.9 - ETA: 50s - loss: 0.3199 - accurac - ETA: 50s - loss: 0.33 - ETA: 48s - loss: 0.3331 - a - ETA: 47s - loss: 0.3324 - accuracy: 0 - ETA: 46s - loss: 0.3326 - accuracy: - ETA: 46s - loss: 0.3341 - accu - ETA: 45s - loss: 0.3305 - ETA: 44s - loss: 0.3342 - accuracy - ETA: 43s - loss: 0.3357 - accuracy: 0.91 - ETA: 43s - loss: 0.3362 - accuracy: 0 - ETA: 43s - lo - ETA: 41s - loss: 0.3304 - accuracy: 0.92 - ETA: 41s - loss: 0.3305 - accuracy: 0. - ETA: 41s - loss: 0.33 - ETA: 36s - loss:  - ETA: 35s - loss:  - ETA: 33s - loss: 0.3435 - accuracy: 0.9 - ETA: 33s - loss: 0.3440 - a - ETA: 26s - loss: 0.3441 - accu - ETA: 25s - loss: 0.3 - ETA: 24s - loss: 0.3440 - accura - ETA: 23s - loss: 0.3447 - accuracy: 0.9 - E - ETA: 20s - loss: 0.3448 - accuracy - ETA: 20s - loss: 0.3451 - accuracy:  - ETA: 19s - loss: 0.3453 - accuracy: 0. - ETA: 19s - loss: 0.3454 - accuracy

781/781 [==============================] - ETA: 0s - loss: 0.3363 - accuracy: 0.91915- ETA: 46s - loss: 0.3007 - accuracy:  - ETA: 46s - loss: 0.3025 - accuracy: 0.933 - ETA: 46s - loss: 0.3033 - accuracy:  - ETA: 46s - loss: 0.3028 - accuracy: 0 - ETA: 46s - loss: 0.3070 - ETA: 45s - loss: 0.3098 - a - ETA: 44s -  - ETA: 42s - loss: 0.3139 - accuracy: 0. - ETA: 41s - loss: 0.3122 - accuracy: 0.9 - ETA: 41s - loss: 0.3140 - accura - ETA: 40s - l - ETA: 36s - loss: 0.3205 - accuracy: 0. - ETA: 35s - loss: 0.3212 - accuracy: 0.924 - ETA: 35s - loss: 0.3212 - accuracy: - ETA: 35s - loss: 0.3213 - ac - ETA: 34s - loss: 0.3258 - accuracy: - ETA: 33s - loss: 0.3254 - accuracy: 0.922 - ETA: 33s - loss: 0.3257 - accuracy: 0.9 - ETA: 33s - loss: 0.325 - ETA: 32s - loss: 0.3273 - accuracy: 0.9 - ETA: 32s - loss: 0.3276 - accuracy: 0. - ETA: 32s - loss: 0.3273 - accuracy: 0 - ETA: 31s -  - ETA: 29s - loss: 0.3 - ETA: 28s - loss: 0.3291 -  - ETA: 27s - loss: 0.3292 - accuracy: 0.9 - ETA: 27s - los

781/781 [==============================] - ETA: 0s - loss: 0.3369 - accuracy: 0.9183 - ETA: 51s - loss: 0.3138 - accuracy: 0 - ETA: 51s - los - ETA: 48s - loss: 0.3153 - ac - ETA: 47s - loss: 0.3165 - accuracy: 0.92  - ETA: 44s - loss - ETA: 42s - loss: 0.3223 - ETA: 38s - loss: 0.3227 - accurac - ETA: 37s - loss: 0.32 - ETA: 36s - loss: 0.3238 - ac - ETA: 35s - loss: 0.3245 - accuracy: 0 - ETA: 35s - loss - ETA: 33s - loss:  - ETA: 28s - loss: 0.3288 -  - ETA: 27s - loss: 0.3310 - acc - ETA: 26s - loss: 0.3306 -  - ETA: 25s - loss: 0.3311 - accuracy - ETA: 25s - loss: - ETA: 23s - loss: 0.3338 - accuracy: 0 - ETA: 23s - loss: 0.3343 - ac - ETA: 22s - loss: 0.3349 - accuracy: - ETA: 21s - loss: 0.3357 - accuracy: 0. - ETA: 21s - loss: 0.3354 - accuracy: - ETA: - ETA: 12s - loss: 0.3350 - accurac - ETA: 12s - loss:  - ETA: 0s - loss: 0.3370 - accura
Epoch 00021: val_accuracy did not improve from 0.88120
781/781 [==============================] - 55s 71ms/step - loss: 0.3369 - accuracy: 

...

781/781 [==============================] - ETA: 0s - loss: 0.2977 - accuracy: 0.9284 - ETA - ETA: 47s - loss: 0.2733 - accu - ETA: 47s - loss: 0.2822 - accuracy: 0.935 - ETA: 47s - loss: 0.2851 - accur - E - ETA: 40s - loss: 0.2842 - accuracy:  - ETA: 40s - loss: 0.2838 - accuracy: - ETA: 39s - loss: 0.2837 - acc - ETA: 38s - loss: 0.2850 - accur - ETA: 38s - loss: 0.2837 - accuracy: 0. - ETA: 37s - loss: 0.2835 - accuracy: 0. - ETA: 37s - loss: 0.2833 - accuracy - ETA: 37s - loss:  - ETA: 35s - lo - ETA: 33s - loss: 0.2876  - ETA: 32s - loss: 0.2882 - ac - ETA: 31s - loss: 0.2894 -  - ETA: 30s - loss: 0.2899  - ETA: 2 - ETA: 23s - loss: 0.2938 - accuracy: 0.92 - ETA: 23s - loss: 0.2939 - accuracy: 0 - ETA: 23s - loss: 0.2938 - accurac - ETA: 22s - loss: 0.2932 - accur - ETA: - ETA: 16s - loss: 0.2949 - ETA: 15s - loss: 0.2946 - accuracy:  - ETA: 14s - loss: 0.2949 - accuracy: 0.9 - ETA: 14s - loss: 0.2949 - acc - ETA: 13s - loss: 0.2950 - accuracy: 0 - ETA: 13s - loss: 0.2950 - ETA: 1

781/781 [==============================] - ETA: 0s - loss: 0.2958 - accuracy: 0.9285 - ETA: 50s - loss: 0.2992 - accuracy: 0.932 - ETA: 50s - loss: 0 - ETA: 49s - ETA: 46s - loss: 0.2863 - ETA: 45s - loss: 0.2946 - - ETA: 44s - loss: 0.2890 - accuracy: 0.93 - ETA - E - ETA: 39s - loss: 0.2891 - - ETA: 37s  - ETA: 35s - loss: 0.2900 - acc - ETA: 34s  - ETA: 27s - loss: 0.2891 - accuracy: 0.93 - ETA: 26s - loss: 0.2894 - accuracy: 0.93 - ETA: 26s - loss: 0.2894 - a - ETA: 25s - loss: 0.2907 - accuracy: 0. - ETA: 25s - loss: 0.2915 - accuracy - ETA: 24s - lo - ETA: 22s - loss: - ETA: 21s - loss: 0.29 - ETA: 19s - loss: 0.2910 - accuracy: 0.9 - ETA: 19s - loss: 0. - ETA: 18s - loss: 0.2904 - accuracy:  - ET - ETA: 15s - loss: 0.2918 - accuracy: 0 - ETA: 14s - loss: 0.2925 - accu - ETA: 14s - loss: 0.2922 - accuracy: 0.930 - ETA: 13s - loss: 0.2923 - a - ETA: 12s - loss: 0.2926 - accurac - ETA: 12s - loss: 0.2925 - accuracy - ETA: 11s - loss: 0.2926 -  - ETA: 10s - loss: 0.2928 - accuracy: 

781/781 [==============================] - ETA: 0s - loss: 0.2921 - accuracy: 0.9308 - ETA: 49s - loss: 0.2721 - accuracy: 0.937 - ETA: 49s - loss: 0.2760 - accurac - ETA: 48s - loss: 0.2802 - accuracy:  - ETA: 48s - loss: 0.2842 - accur - ETA: 47s - loss: 0.2914 - accuracy - ETA: 47s - loss: 0.2954 - accuracy: 0. - ETA: 47s - loss: 0.2935 - accurac - ETA: 46s - loss: 0.2927 - accura - ETA: 46s - loss: 0.2860 - accuracy: 0.936 - ETA: 46s - loss: 0.2851 - accuracy: 0 - ETA: 45s - loss: 0.2821 - accuracy: 0. - ETA: 45s - loss: 0.2829 - accuracy: - ETA: 45 - ETA: 42s - loss: 0.2820 - ac - ETA: 41s - loss: 0.2825 - accuracy:  - ETA: 41s - loss: 0.2831 - accurac - ETA: 41s - loss: 0.2849 - accurac - ETA: 40s - loss: 0.2837 - accuracy: 0.93 - ETA: - ETA: 37s - loss: 0.2829 - accuracy: 0 - ETA: 37s - loss: 0.2830 - accu - ETA: 36s - loss: 0.2846 - accura - ETA: 36s - loss: 0.2833 - accuracy: 0. - ETA: 35s - loss: 0.2844 -  - ETA: 34s - loss: 0.2845 - accuracy: 0. - ETA: 34s - loss - ETA: 32s 

781/781 [==============================] - ETA: 0s - loss: 0.2888 - accuracy: 0.9327 - ETA: 53s - loss: 0.2670 - accuracy: - ETA: 51s - l - ETA: 49s - loss:   - ETA: 44s - loss: 0.2818 - accuracy: - ETA: 43s - loss: 0.2814 - accu - ETA: 43s - loss: 0.2796 - accuracy: - ETA: 36s - loss: 0.2807 - acc - ETA: 35s - loss: 0.2808 - accuracy: 0 - ETA: 35s - loss: 0.2811 - accuracy: 0.93 - ETA: 35s - loss: 0.2812 -  - ETA: 34s - loss - ETA: 32s - loss: 0.2828 - accuracy: 0.9 - ETA: 32s - loss: 0.2832 - accu - ETA: 31s - loss: 0.2825 - accur - ETA: 3 - ETA: 25s - loss: 0.2834 - accuracy: 0.93 - ETA: 24s - loss: 0.2831 - accur - ETA: 24s - loss: 0.2825 - accuracy: - ETA: 23s - loss: 0.2818 - accuracy: 0. - ETA: 23s - loss: 0.2820 - accu  - ETA: 19s - loss: 0.2843 - accuracy:  - ETA: 19s - loss: 0.2845 - accurac - ETA: 18s - loss: 0.2843 - acc - ETA: 17s - loss: 0.2851 - accuracy: 0.9 - ETA: 17s - loss: 0.2849 - E
Epoch 00094: val_accuracy did not improve from 0.89220
781/781 [===================

781/781 [==============================] - ETA: 0s - loss: 0.2872 - accuracy: 0.9339 - ETA: 49s - loss: 0.2971 - accuracy: 0.927 - ETA: 49s - loss: 0.2932 - accuracy: 0 - ETA: 48s - loss: 0.2844 - accuracy: 0.92 - ETA: 48s - loss: 0.2844 - accura - ETA: 48s - loss: 0.2750 - accuracy:  - ETA: 47s - loss: 0.2729 - accuracy: 0 - ETA:  - ETA: 44s - loss: 0.2815 - acc - ETA: 43s - loss: 0.2852 - ETA: 43s - loss: 0.28 - ETA: 41s - loss: 0.2863 - accuracy:  - ETA: 41s - loss: 0.2861 - accuracy: 0.93 - ETA: 41s - los - ETA: 39s - loss: 0.2860 - accuracy: 0 - ETA: 39s - loss: 0.2862 - accurac - ETA: 38s - - ETA:  - ETA: 31s - loss: 0.2819 - a - ETA: 30s - loss: 0.2831 - ac - ETA: 29s - loss: 0.285 - ETA: 27s - loss:  - ETA: 26s - loss: 0.2848 - accuracy: 0 - ETA: 25s - loss: 0.2848 - accuracy: 0.934 - ETA: 25s - loss: 0.2852 - accu - ETA: 24s - loss: 0.2862 - accuracy: - ETA: 24s - loss: 0.2863 - accuracy: - ETA: 24s - loss: 0.2868 - accuracy: - ETA: 23s - loss: 0.2875 - accurac - ETA: 23s - lo

In [22]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7780), started 12:16:39 ago. (Use '!kill 7780' to kill it.)

In [25]:
from keras.models import load_model
best_model = load_model('my_best_model.epoch67-accuracy0.89.hdf5')

In [26]:
# Test the model
score = best_model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 4s 12ms/step - loss: 0.4415 - accuracy: 0.8922
Test loss: 0.441528856754303
Test accuracy: 0.8921999931335449


In [27]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.0003,decay=1e-6),
              metrics=['accuracy'])

In [29]:
model.fit(img_gen,
          steps_per_epoch=steps,
          epochs=30,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint,tensorboard_callback])

Epoch 1/30
781/781 [==============================] - ETA: 0s - loss: 0.2635 - accuracy: 0.9411  - ETA: 53s - loss: 0.2804 - accurac - ETA: 52s - loss: - ETA: 42s - loss: 0.2658 - accuracy: 0.939 - ETA: 42s - loss: 0.2650 - ETA: 41s - loss: 0.2643 - - ETA: 39s - loss: 0.2652 - accurac - ETA: 39s - loss: 0.2 - ETA: 38s - loss: 0.2653 - ac - ETA: 37s - loss:  - ETA: 35s - loss: 0.2644 - accuracy - ETA: 34s - loss: 0.26 - ETA: 33s - loss: 0.2665 - accuracy - ETA: - ETA: 30s - loss: 0.2658 - accuracy: 0 - ETA: 30s - loss: 0.26 - ETA:  - ETA: 26s  - ETA: 24s - loss: 0.2660 - accuracy: 0.9 - ETA: 23s - loss: 0.2660 - - ETA: 22s - loss: 0.2655 - accurac - ETA: 22s - loss: 0.2661 - accurac - ETA: 21s - loss: 0.2654 - - ETA: 20s - loss: 0.2656 - accuracy: 0.940 - ETA: 20s - loss: 0.2655 - - ETA: 19s - loss: 0.2647 -  - ETA: 18s - loss: 0.2645 - accuracy: 0. - ETA: 18s - loss: 0.2644 - accuracy: 0. - ETA: 9s - loss: 0.2633 - ac - ETA: 9s - loss: 0.263 - ETA: 6s - loss: 0.2639 - ac - ETA: 6s - lo

781/781 [==============================] - ETA: 0s - loss: 0.2464 - accuracy: 0.9451 - - ETA: 37s - loss: 0.2415 - accur - ETA: 36s - loss: 0.2420 - accuracy: 0.94 - ETA: 36s - loss: 0.2419 - accuracy: 0.94 - ETA: 36s - los - ETA: 31s - loss: 0.2414 - ETA: 27s - loss: 0.2431 - accuracy: 0.944 - ETA: 27s  - ETA: 24s - loss: 0.2434 - accuracy: 0.944 - ETA: 24s - loss: 0.2433  - ETA: 23s - loss: 0.2438 - accur - ETA: 22s - loss: 0.2447 - accuracy: - ETA: 22s - loss: 0.2444 - accuracy: 0.9 - ETA: 22s - loss: 0.2444 - accuracy: 0 - ETA: 21s - loss: 0.2444 - accuracy - E  - ETA: 12s - loss: 0.2443 - ac - ETA: 11s - loss: 0.2446 - accu - ETA: 11s - loss: 0.2445 - accuracy: 0. - ETA: 10s - loss: 0.2447 - accuracy: 0. - ETA: 10s - loss: 0.2447 - accuracy: 0.9 - - ETA: 4s - loss: 0.2464 - accu - ETA: 3s - loss: 0.2464 - accu - ETA: 3s - loss: 0.2466 - accura - ETA: 3s - loss: 0.2466 - accuracy: 0.94 - ETA: 3s - loss: 0.2466 - accuracy - ETA: 2s - loss: 0.2468 - accuracy: 0.94 - ETA: 2s - loss: 0

781/781 [==============================] - ETA: 0s - loss: 0.2441 - accuracy: 0.9461 - ETA: 50s - loss: 0.2491 -  - ETA: 49s - loss: 0.2418 - accuracy: 0 - ETA: 49s - loss: 0.2355 - acc - ETA: 48s - loss: 0.2347 - accuracy: 0.949 - ETA: 48s - loss: 0.2336 - accura - E - ETA: 44s - ETA: 42s - loss: 0.2459 - accuracy: 0.9 - ETA: 42s - loss: 0.2477 - accuracy: 0.9 - ETA: 42s - loss - ETA:  - ETA: 37s - loss: 0.2462 - accuracy:  - ETA: 37s - loss: 0.2465 - accurac - ETA: 36s - loss:  - ETA: 35s - loss: 0.2475 - a - ETA: 34s - loss: 0.2453 - - ETA: 32s - loss: 0.2458 - accuracy: 0.94 - ETA: 32s - ETA: 30s - loss: 0.2458 - - ETA: 29s - loss: 0.2450 - accuracy: 0.94 - ETA: 29s - loss: 0.2451 - accur - ETA: 28s - loss:  - ETA: 27s - loss: 0.2469 - accuracy: 0.9 - ETA: 26s - loss: 0.24 - ETA: 25s - loss - ETA: 23s - loss: 0.2445 -  - ETA - ETA: 20s - loss: 0.2436 - accurac - ETA: - ETA: 17s - loss: 0.2441 -  - ETA: 15s - loss: 0.2433 - accuracy: 0.9 - ETA: 15s - loss: 0.2432 - accuracy: 0.9  - 

781/781 [==============================] - ETA: 0s - loss: 0.2348 - accuracy: 0.94798- ETA: 48s - loss: - ETA: 47s  - ETA: 45s - loss: 0.2298 - accuracy: 0.9 - ETA: 44 - ETA: 43s - loss: 0.2324 - accuracy: 0 - ETA: 42s - loss: 0.2336 - accuracy: 0. - ETA: 42s - loss: 0.2338 - accu - ETA: 41s - loss: 0.2360 - acc - ETA: 40s - loss: 0.2361 - accu - ETA: 39 - ETA: 37s - - ETA: 35s - loss: 0.2330 - accu - ETA: 34s - loss: 0.2320 - accurac - ETA: 34s - loss: 0.2302 - accuracy - ETA: 33s - loss: 0.2304 - accuracy: 0.9 - ETA: 33s - loss: 0. - ETA: 32s - loss: 0.2324 - accuracy: 0.9 - ETA: 32s -  - ETA: 30s - loss: 0.2306 - accur - ETA: 29s - l - ETA: 27s - loss: 0.2314 - accu - ETA: 26s - loss: 0.2322 - accurac - ETA: 26s - loss: 0.2320 - accur - ETA: 22s - loss: 0.2316 - accuracy:  - ETA: 21s - loss: 0.2322 - ETA: 20s - loss: 0.2337 - accuracy - ETA: 20s - loss:  - ETA: 18s - lo - ETA: 16s - loss: 0.2349 - accu - ETA: 15s - loss: - ETA: 11s - loss: 0.2348 - accuracy: 0.94 - ETA: 11s - loss: 

781/781 [==============================] - ETA: 0s - loss: 0.2355 - accuracy: 0.9471 - ETA: 45s - loss: 0.2229  - ETA: 44s - loss: 0.2250 - accuracy: 0. - ETA: 44s - loss: 0.2244 - ac - ETA: 43s - loss: 0.2286 - accuracy:  - ETA: 42s - loss: 0.2285 - accur - ETA: 42s - loss: 0.2285 - accura - ETA: 41s - loss: 0.2283 - accuracy: 0.9 - ETA: 41s - loss: 0.2286 - accuracy:  - ETA: 41s - loss: 0.2266 - acc - ETA: 40s - loss: 0.2257 - accuracy: 0. - ETA: 39s - loss: 0.2259 - accuracy: 0.948 - ETA: 39s - loss: 0.2262 - accuracy: 0.9 - ETA: 39s - loss: 0.2270 - accuracy: 0 - E - ETA: 36s - loss: 0.2264  - ETA: 35s - loss: 0.2270 - ETA: 34s - loss: 0.22 - ETA: 32s - loss: 0.2268 - accur - ETA: 32s - loss: 0.2265 - accurac - ETA: 31s - loss: 0.2268 - accu - ETA: 30s - loss: 0.2283 - accuracy: 0.94 - ETA: 30s - loss: 0.2278 - accuracy: 0.949 - ETA: 30s - loss: - ETA: 28s - loss: 0.2271 - accuracy: 0 - ETA: 28s - loss: 0.228 -  - ETA: 21s - loss: 0.2309 - accurac - ETA: 21s -  - ETA: 19s - loss: 0

781/781 [==============================] - ETA: 0s - loss: 0.2324 - accuracy: 0.9475 - ET - ETA: 42s - loss: 0.225 - ETA: 41s - loss: 0.2296 - accurac - ETA: 41s - loss: 0.2306 - acc - ETA: 40s - loss: 0.2304 - accu - ETA: 39s - loss: 0.2298 - accuracy - ETA: 38s - loss: 0.229 - ETA: 37s - loss: 0.2289 - accuracy: 0.948 - ETA: 37s - loss: 0.2287 - ETA: 36s - loss: 0.2308 - accur - ETA: 35s - loss: 0.2306 - accura - ETA: 34s - loss - ETA: 23s - loss: 0.2290 - accuracy: 0.9 - ETA: 23s - loss: 0.2292 - accura - ETA: 22s  - ETA: 20s - loss: 0.2292 - accuracy: 0.949 - ETA: 20s -  - ETA: - ETA: 16s - loss: 0.2288 - accuracy: 0.94 - ETA: 15s - loss: 0.228 - ETA: 14s - loss: 0.2293 - - ETA: 13s - loss: 0.2298 - accurac - ETA: 12s - loss: 0.2297 - accuracy: 0.948 - ETA: 12s - loss: 0.2295 - accuracy: 0 - ETA: 12s - ETA: 10s - loss:  - ETA: 7s - loss: 0.2303 - accuracy - ETA: 7s - loss: 0.2304 -  - ETA: 7s - loss: 0.2302 - accuracy: 0. - ETA: 6s - loss: 0.2303  - ETA: 6s - loss: 0.2309 - accu - 

In [30]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7780), started 12:48:35 ago. (Use '!kill 7780' to kill it.)

In [31]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.0002,decay=1e-6),
              metrics=['accuracy'])

In [33]:
model.fit(img_gen,
          steps_per_epoch=steps,
          epochs=25,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint,tensorboard_callback])

Epoch 1/25
781/781 [==============================] - ETA: 0s - loss: 0.2124 - accuracy: 0.9541 - ETA: 44s - loss: 0.2047 - accuracy: 0.95 - ETA: 44s - loss: 0.2040 - accuracy: 0.95 - ETA: 44s  - ETA: 42s - ETA: 40s - loss: 0.2075 - accuracy - ETA: 40s - loss: 0.2082 - accuracy - ETA: 39s - loss: - ETA: 37s - loss - ETA: 36s - los - ETA: 28s - loss: 0.2082 - accurac - ETA: 25s - loss: 0.2100 - accuracy: - ETA: 24s - loss: 0.210 - ETA: 23s - loss: 0.2103 - accuracy - ETA: 22s - loss: 0. - ETA: 21s - - ETA: 19s - loss: 0.2115 - accura - ETA: 18s - loss: 0.2124 - accuracy: - ETA: 18s - l - ETA: 16s - loss: 0.2124  - ETA: 14s - loss: 0.2114 - accuracy: 0.954 - ETA: 14s - loss: 0.2114 - accurac - ETA: 4s - loss: 0.2115 - ac
Epoch 00001: val_accuracy did not improve from 0.89850
781/781 [==============================] - 63s 72ms/step - loss: 0.2124 - accuracy: 0.9541 - val_loss: 0.4724 - val_accuracy: 0.8944
Epoch 2/25
781/781 [==============================] - ETA: 0s - loss: 0.2080 - accu

781/781 [==============================] - ETA: 0s - loss: 0.2055 - accuracy: 0.9560 - ETA: 47s - loss: 0.1987 - accuracy - ETA: 46s - loss:  - ETA: 45s - loss: 0.2007 - accurac - ETA: 44s - loss: 0.1994 - accuracy: 0. - ETA: 44s - ETA: 39s - loss: 0.1977 - accuracy: 0.95 - ETA: 39s - loss: - ETA: 37s - loss: 0.1985 - accuracy:  - ETA: 37s -  - ETA: 35s - loss: 0.1983 - a - E - ETA: 32s - loss: 0.19 - ETA: 30s - loss: 0.1988 - accuracy: 0 - ETA: 30s - loss: 0.1983 - accuracy: 0 - ETA: 29s - loss: 0.1981 - accuracy: - - ETA: 26s - loss:  - ETA: 25s - loss: 0.2000 - accuracy - ETA: 24s - loss: 0.2005 - accuracy: 0. - ETA: 24s - loss: 0.2007 - accuracy: 0.958 - ETA: 24s - loss: 0.2006 - accuracy: 0 - ETA: 24s - loss: 0.2013 - accuracy: 0.958 - ETA: 23s - loss: 0.2014 - accur - ETA: 23s - loss: 0.2023 - acc - ETA: 22s - loss: 0 - ETA: 20s - loss: 0.2025 - accuracy: 0.95 - ETA: - - ETA: 15s - loss: 0.2037 - accuracy: 0.9 - ETA: 15s - loss: 0.2038 - accuracy: - ETA: 14s - loss: 0.2040 - a - 

781/781 [==============================] - ETA: 0s - loss: 0.2012 - accuracy: 0.9582 -  - ETA: 46s - ETA: 43s - loss: 0.1961 - accuracy - ETA: 43s - loss: 0.1957 -  - ETA: 42s - loss: 0.1969  - ETA: 41s - loss: 0.1970 - ac - ETA: 37s - loss: 0.1964 - accuracy:  - ETA: 33s - lo - ETA: 31s - loss: 0.1988 - a - ETA: 30s - loss: 0.1975 - accurac - ETA: 29s - loss: 0.1978 - ac - ETA: 28s - loss: 0.1987 - a - ETA: 27s - lo - ETA: 25s - loss: 0. - ETA: 24s - loss: 0.1993 - accuracy:  - ETA: 23 - ETA: 21s - loss: 0.1995 - a - ETA: 20s - loss: - ETA: 18s - loss: 0.1999 - accuracy: 0 - ETA: 18s - loss: 0.1995 - - ETA: 17s - lo - ETA: 15s - loss: 0.2010  - ETA: 11s - loss: 0.2013 - accuracy: 0.9 - ETA: 11s - loss: 0.2012 - accu - ETA: 10s - loss: 0.2009 - ac - ETA: 8s - loss: 0.2010 - accu - E - ETA: 3s - loss: 0.2011 - accura - E - ETA: 1s - loss: 0.2009 -  - ETA: 1s - loss: 0.2008 - accu - ETA: 0s - l
Epoch 00012: val_accuracy did not improve from 0.90090
781/781 [==============================

781/781 [==============================] - ETA: 0s - loss: 0.1964 - accuracy: 0.9588 - ETA: 50s - loss: 0.17 - ETA: 48s - loss: 0.1850 - accuracy - ETA: 47s - loss: 0 - ETA: 45s - loss: 0.1953 - accuracy:  - ETA: 45s - loss: 0.1952 - accuracy: 0. - ETA: 45s - loss: 0.1934 - accuracy: 0.95 - ETA: 45s - loss: 0.1939 - accuracy: 0 - ETA: 44s - loss: 0.1942 - accuracy: 0. - ETA: 44s - loss: 0.1943 - accuracy:  - ETA: 44 - ETA: 42s - loss: 0.1905 - accuracy: 0.95 - ETA: 41s - loss: 0.1908 - a - ETA: 37s - loss: 0.1935 - accurac - ETA: 37s - loss: 0.1933 - accuracy: 0.95 - ETA: 37s - loss: 0.1932 - accuracy: - ETA: 36s - loss: 0.1936 - accuracy: 0. - ETA: 36s - loss:  - ETA: 34s - loss: 0.1914 - accuracy: 0. - ETA: 34s - loss: 0.1912 - accuracy: 0.9 - ETA: 31s - loss - ETA: 26s - loss: 0.1 - ETA: 25s - loss: 0.1979 - - ETA: 24s - loss: 0 - ETA: 22s - l - ETA: 20s - loss: 0.1985 - accuracy: 0.958 - ETA: 20s - los - ETA: 18s - loss: 0.19 - ETA: 17s  - ETA: 15s - loss: 0.1979 - accuracy: 0 - ET

781/781 [==============================] - ETA: 0s - loss: 0.1932 - accuracy: 0.9594 - ETA: 50s - loss: 0.1916 - accuracy: 0.9 - ETA: 50s - loss: 0.1908 - accuracy: 0.956 - ETA: 50s - loss: 0.1918 -  - ETA: 48s - loss: 0.1929 - accura - ETA: 45s - loss: 0.1840 - ETA: 44s - loss: 0.1829 - accuracy - ETA: 43s - loss: 0.1826 - accuracy: 0 - ETA: 40s - loss: 0.1887 - accur - ETA: 39s - loss: 0.1896 - accuracy: 0.96 - ETA: 39s -  - ETA: 37s - loss: 0.1911 - accuracy: 0. - ETA: 36s - loss: 0.1910 - accuracy: 0.961 - ETA: 36s - loss: 0.1907 - accuracy: 0.96 - ETA: 36s - loss: 0.1906 - ac - ETA: 35s - loss: 0.1901 - a - ETA: 34s - loss: 0.1908 - accurac - ETA: 33 - ETA: 31s - loss: 0.1921 - accur - ETA: 3 - ETA:  - ETA: 25s - loss: 0.1917 - ETA: 24s - loss: 0.1918 - accuracy: 0. - ETA: 24s - loss: 0.1917 - accuracy - ETA: 23s - loss: 0.1917 - accurac -  - ETA: 20s - loss: 0.1921 - accuracy - ETA: 20s - loss: 0.1923 - accuracy: 0.9 - ETA: 19s - loss: 0.1921 - accur - ETA: 19s - loss: 0.1925 - a

In [34]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7780), started 13:14:04 ago. (Use '!kill 7780' to kill it.)

In [35]:
best_model = load_model('my_best_model.epoch15-accuracy0.90.hdf5')
score = best_model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 5s 12ms/step - loss: 0.4338 - accuracy: 0.9043
Test loss: 0.43384525179862976
Test accuracy: 0.9042999744415283
